In [1]:
import pandas as pd

doc_dict = {}
id_dict = {}
word_count_dict = {}
country_set = set()

# pulls text IDs, country codes, document types, and word counts from the excel sheet,
# using it to divide the documents into a dictionary by ID
sources_df = pd.read_excel("./text/sampleSources.xlsx", sheet_name="texts")
for text_id, (country_code, doc_type), word_count in [(l[0], tuple(l[1].split()), l[2]) for l in sources_df[["textID", "country|genre", "# words"]].values.tolist()]:
    with open(f"./text/w_{country_code.lower()}_{doc_type.lower()}.txt", 'r',
              encoding="utf-8") as file:
        # add each text_id to id_dict
        if f"{country_code}_{doc_type}" not in id_dict:
            id_dict[f"{country_code}_{doc_type}"] = [text_id]
        else:
            id_dict[f"{country_code}_{doc_type}"].append(text_id)
        # makes country code set
        country_set.add(country_code)
        # finds correct text_id and adds every line in the document to the dictionary
        IS_DOC = False
        lines = file.readlines()
        for i, line in enumerate(lines):
            if line.strip().startswith(f"##{text_id}"):
                IS_DOC = True
            elif line.strip().startswith("##"):
                IS_DOC = False
            if IS_DOC:
                if text_id not in doc_dict:
                    doc_dict[text_id] = [w.lower() for w in line.split()]
                else:
                    doc_dict[text_id] += [w.lower() for w in line.split()]
        # adds word count to dictionary
        word_count_dict[text_id] = word_count

In [2]:
import copy
from collections import Counter

# make a counter for every word in the corpus
vocab = Counter({})
vocab['<UNK>'] = 0
for doc in doc_dict.values():
    for word in doc:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

# make a dictionary of sets for every country and record every word used by each country
# also make a word count for every country
vocab_sets = {country_code:set() for country_code in country_set}
country_word_counts = Counter({country_code:0 for country_code in country_set})
for country_code in country_set:
    for text_id in id_dict[f"{country_code}_B"] + id_dict[f"{country_code}_G"]:
        for word in doc_dict[text_id]:
            vocab_sets[country_code].add(word)
        country_word_counts[country_code] += word_count_dict[text_id]

# make new vocabulary sets, removing words that appear in less than
# 25% and 50% of the countries datasets
vocab_25 = vocab.copy()
vocab_50 = vocab.copy()
for word in vocab:
    COUNTRY_COUNT = 0
    for country_code in country_set:
        if word in vocab_sets[country_code]:
            COUNTRY_COUNT+=1
    if COUNTRY_COUNT / len(country_set) < 0.25:
        del vocab_25[word]
    if COUNTRY_COUNT / len(country_set) < 0.5:
        del vocab_50[word]

# Replace any words that appear in less than 25% of the countries’ datasets with the <UNK> token
doc_dict_25 = copy.deepcopy(doc_dict)
for text_id, doc in doc_dict_25.items():
    for i, word in enumerate(doc):
        if word not in vocab_25:
            doc_dict_25[text_id][i] = '<UNK>'
            vocab_25['<UNK>'] += 1

# Replace any words that appear in less than 50% of the countries’ datasets with the <UNK> token
doc_dict_50 = copy.deepcopy(doc_dict)
for text_id, doc in doc_dict_50.items():
    for i, word in enumerate(doc):
        if word not in vocab_50:
            doc_dict_50[text_id][i] = '<UNK>'
            vocab_50['<UNK>'] += 1

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# make a dataframe with each version of the dataset with their country labels
texts = []
texts_25 = []
texts_50 = []
country_labels = []

for country_code in country_set:
    for text_id in id_dict[f"{country_code}_B"] + id_dict[f"{country_code}_G"]:
        texts.append(" ".join(doc_dict[text_id]))
        texts_25.append(" ".join(doc_dict_25[text_id]))
        texts_50.append(" ".join(doc_dict_50[text_id]))
        country_labels.append(country_code)

data = {
    'texts': texts,
    'texts_25': texts_25,
    'texts_50': texts_50,
    'country_labels': country_labels
}

df = pd.DataFrame(data)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['texts'])
X_25 = vectorizer.fit_transform(df['texts_25'])
X_50 = vectorizer.fit_transform(df['texts_50'])
y = df['country_labels']

In [4]:
def probability_breakdown(model, X_test, y_test):
    # makes a dictionary of probability Counters for every country label for each country's documents
    most_probable_country = {c:Counter({c:0 for c in country_set}) for c in country_set}
    for test_num, probabilities in enumerate(model.predict_proba(X_test)):
        for i, probability in enumerate(probabilities):
            most_probable_country[y_test.tolist()[test_num]][model.classes_[i]] += probability  

    return most_probable_country

In [5]:
def average_tries(model, X_test, y_test, print_output=False):
    """
    Takes the model and testing data.
    Returns average tries needed to return the right label,
    average tries per country, and number of tests per country.
    """
    
    # Matches each country label to its probability in a dataframe
    df = pd.DataFrame(model.predict_proba(X_test).tolist(), columns=model.classes_.tolist())

    # Sorts each probability distribution for highest probability
    # and records the number of iterations needed to get to the right label
    country_try_count = {c:0 for c in country_set}
    country_test_count = Counter(y_test)
    for i in range(len(df)):
        for try_count, (country_code, _) in enumerate(sorted(df.iloc[i].to_dict().items(), key=lambda item: item[1], reverse=True)):
            if country_code == y_test.tolist()[i]:
                country_try_count[country_code] += try_count+1
                break
    
    # Averages the country try count by number of tests
    country_try_count = {country:country_try_count[country]/country_test_count[country] for country in country_try_count}
    # Computes overal average try count
    avg_tries = sum(country_try_count.values())/len(country_try_count)
    
    # Either returns or prints the results
    if print_output:
        for country, try_count in sorted(country_try_count.items(), key=lambda item: item[1]):
            print(country, f"{try_count:.1f} tries", f"({country_test_count[country]} tests)")
        print("Average number of tries:", avg_tries)
    else:
        return avg_tries, country_try_count, country_test_count

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import random

def average_performance(model, X_dataset, y_dataset = y, random_states=10, undersampling=False, scrambling=False):
    """
    Takes the model, dataset, and number of random test splits to test.
    Prints average accuracy and tries between all test splits of the model.
    """
    accuracies = []
    tries = []
    avg_country_try_count = {c:0 for c in country_set}
    total_country_test_count = {c:0 for c in country_set}
    avg_most_probable_country = {c:Counter({c:0 for c in country_set}) for c in country_set}
    
    if scrambling:
        random.seed = 3
        y_scrambled = y_dataset.tolist()
        random.shuffle(y_scrambled)
        y_dataset = pd.Series(y_scrambled)

    # trains models on the specified number of random splits
    for r in range(random_states):
        X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, test_size=0.2, random_state=r, stratify=y_dataset)
        # undersampling
        if undersampling:
            X_train, y_train = RandomUnderSampler(random_state=3).fit_resample(X_train, y_train)
        model.fit(X_train, y_train)
        # computes accuracies
        y_pred = model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        # computes overall and country try counts
        avg_tries, country_try_count, country_test_count = average_tries(model, X_test, y_test)
        tries.append(avg_tries)
        # computes similar countries
        most_probable_country = probability_breakdown(model, X_test, y_test)
        for country in country_try_count:
            avg_country_try_count[country] += country_try_count[country]
            total_country_test_count[country] += country_test_count[country]
            avg_most_probable_country[country] += most_probable_country[country]
        print(".",end=" ")
    print()
    
    print("Average Accuracy:", np.mean(accuracies))
    print("Averaged Average tries:", np.mean(tries))
    # Sorts country try counts and prints them in order
    avg_country_try_count = {country:avg_country_try_count[country]/(random_states) for country in avg_country_try_count}
    for country, try_count in sorted(avg_country_try_count.items(), key=lambda item: item[1]):
        most_similar_five = " ".join([country for country, _ in avg_most_probable_country[country].most_common(5)])
        print(f"{country}: {try_count:4.1f} tries", f"({total_country_test_count[country]:3d} tests) | {most_similar_five}")

In [7]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression model with class weights balanced
model = LogisticRegression(max_iter=2000, class_weight='balanced')
average_performance(model, X)
average_performance(model, X_25)
average_performance(model, X_50)

. . . . . . . . . . 
Average Accuracy: 0.2990909090909091
Averaged Average tries: 7.252184875226604
GB:  2.5 tries (684 tests) | GB US AU CA IE
US:  2.7 tries (649 tests) | US GB CA AU IN
AU:  4.5 tries (266 tests) | AU GB US CA IN
IE:  5.4 tries (180 tests) | IE GB US AU PH
BD:  5.5 tries ( 80 tests) | BD GB US PK IE
CA:  5.5 tries (252 tests) | US GB CA AU IE
IN:  6.0 tries (176 tests) | GB IN US LK CA
NZ:  6.2 tries (133 tests) | GB US NZ AU CA
LK:  6.5 tries ( 67 tests) | LK GB US AU CA
KE:  6.9 tries ( 75 tests) | KE GB US AU IN
PK:  7.2 tries ( 97 tests) | US PK GB AU IN
GH:  8.2 tries ( 65 tests) | GH GB US AU KE
SG:  8.4 tries ( 86 tests) | US GB SG IN NZ
TZ:  9.0 tries ( 49 tests) | GB KE US TZ IN
ZA:  9.4 tries ( 74 tests) | US GB AU ZA CA
MY:  9.6 tries ( 83 tests) | GB US MY SG BD
HK:  9.7 tries ( 72 tests) | US GB HK MY AU
NG:  9.8 tries ( 70 tests) | GB US NG IN CA
JM:  9.9 tries ( 70 tests) | GB US CA ZA JM
PH: 12.3 tries ( 72 tests) | GB US AU CA SG
. . . . . . . . . . 

In [8]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier with class weights balanced
rf_model = RandomForestClassifier(random_state=3, class_weight='balanced')
average_performance(rf_model, X)
average_performance(rf_model, X_25)
average_performance(rf_model, X_50)

. . . . . . . . . . 
Average Accuracy: 0.3242424242424242
Averaged Average tries: 6.431463450685949
US:  1.5 tries (649 tests) | US GB CA AU IN
GB:  1.5 tries (684 tests) | GB US AU CA IE
AU:  3.2 tries (266 tests) | GB US AU CA NZ
IE:  3.5 tries (180 tests) | GB US IE AU CA
CA:  3.7 tries (252 tests) | US GB CA AU IN
BD:  4.3 tries ( 80 tests) | US GB BD IN AU
LK:  4.5 tries ( 67 tests) | LK GB US IN AU
IN:  4.9 tries (176 tests) | US GB IN AU CA
NZ:  5.3 tries (133 tests) | GB US NZ AU CA
NG:  7.1 tries ( 70 tests) | US GB NG AU CA
JM:  7.1 tries ( 70 tests) | GB US JM CA AU
GH:  7.5 tries ( 65 tests) | US GB GH AU CA
PK:  8.1 tries ( 97 tests) | GB US PK CA AU
MY:  8.3 tries ( 83 tests) | GB US MY AU CA
HK:  8.7 tries ( 72 tests) | US GB CA AU HK
KE:  8.7 tries ( 75 tests) | GB US AU CA KE
SG:  9.4 tries ( 86 tests) | US GB AU SG CA
TZ: 10.1 tries ( 49 tests) | GB US IE CA AU
ZA: 10.2 tries ( 74 tests) | GB US AU CA ZA
PH: 11.0 tries ( 72 tests) | GB US AU CA IN
. . . . . . . . . . 

In [9]:
from sklearn.neural_network import MLPClassifier

# Multi-layer Perceptron Model
mlp_model = MLPClassifier(random_state=3)
average_performance(mlp_model,X)
average_performance(mlp_model,X_25)
average_performance(mlp_model,X_50)

. . . . . . . . . . 
Average Accuracy: 0.3145454545454546
Averaged Average tries: 7.185977892927338
GB:  1.9 tries (684 tests) | GB US AU CA IE
US:  2.6 tries (649 tests) | US GB AU CA NZ
AU:  3.4 tries (266 tests) | GB US AU CA NZ
CA:  4.3 tries (252 tests) | US GB CA AU IE
IN:  4.9 tries (176 tests) | GB US IN AU CA
IE:  5.8 tries (180 tests) | GB US IE AU CA
NZ:  5.9 tries (133 tests) | GB US NZ AU IN
LK:  6.7 tries ( 67 tests) | GB LK AU US CA
TZ:  7.0 tries ( 49 tests) | GB US CA AU IE
JM:  7.3 tries ( 70 tests) | GB US AU CA JM
KE:  7.3 tries ( 75 tests) | GB US AU NZ CA
BD:  7.5 tries ( 80 tests) | GB US AU BD IE
HK:  8.3 tries ( 72 tests) | US GB HK AU CA
SG:  8.4 tries ( 86 tests) | US GB AU SG CA
PH:  8.6 tries ( 72 tests) | US GB AU CA PH
PK:  8.9 tries ( 97 tests) | GB US PK CA AU
NG: 10.0 tries ( 70 tests) | US GB AU NG CA
GH: 11.2 tries ( 65 tests) | GB US AU GH CA
ZA: 11.4 tries ( 74 tests) | US GB AU CA TZ
MY: 12.2 tries ( 83 tests) | GB US AU CA IE
. . . . . . . . . . 

In [10]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression model with undersampling
model = LogisticRegression(max_iter=2000)
average_performance(model, X, undersampling=True)
average_performance(model, X_25, undersampling=True)
average_performance(model, X_50, undersampling=True)

. . . . . . . . . . 
Average Accuracy: 0.1433333333333333
Averaged Average tries: 7.757905472518357
BD:  4.9 tries ( 80 tests) | BD PK MY KE CA
LK:  6.6 tries ( 67 tests) | LK PH GH TZ JM
TZ:  6.8 tries ( 49 tests) | KE TZ GH IE ZA
KE:  6.8 tries ( 75 tests) | KE TZ JM GH ZA
SG:  7.1 tries ( 86 tests) | SG PH MY TZ ZA
GH:  7.1 tries ( 65 tests) | GH KE PH TZ CA
PK:  7.5 tries ( 97 tests) | PK IN GH TZ NG
IE:  7.5 tries (180 tests) | IE PH TZ GB US
NZ:  7.8 tries (133 tests) | NZ PH GB LK AU
HK:  7.9 tries ( 72 tests) | HK MY TZ NZ PH
JM:  8.1 tries ( 70 tests) | JM CA ZA TZ US
GB:  8.1 tries (684 tests) | GB CA AU US PH
AU:  8.2 tries (266 tests) | AU GB CA NG PH
US:  8.2 tries (649 tests) | US CA GB PH AU
MY:  8.3 tries ( 83 tests) | MY BD SG GB GH
CA:  8.3 tries (252 tests) | CA US AU GB IE
ZA:  8.7 tries ( 74 tests) | ZA PH GB CA IE
IN:  8.9 tries (176 tests) | IN KE GB PH LK
PH:  9.1 tries ( 72 tests) | SG KE TZ PH IE
NG:  9.2 tries ( 70 tests) | NG PH TZ GB LK
. . . . . . . . . . 

In [11]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier with undersampling
rf_model = RandomForestClassifier(random_state=3)
average_performance(rf_model, X, undersampling=True)
average_performance(rf_model, X_25, undersampling=True)
average_performance(rf_model, X_50, undersampling=True)

. . . . . . . . . . 
Average Accuracy: 0.21303030303030304
Averaged Average tries: 6.906405632145004
BD:  3.9 tries ( 80 tests) | BD IN PK NZ CA
LK:  4.5 tries ( 67 tests) | LK IN GB PH CA
IE:  5.3 tries (180 tests) | IE GB US PH CA
JM:  5.7 tries ( 70 tests) | JM CA PH IN ZA
TZ:  6.0 tries ( 49 tests) | TZ ZA KE GH PK
CA:  6.1 tries (252 tests) | CA US GB PH AU
GH:  6.5 tries ( 65 tests) | GH TZ ZA KE GB
NZ:  6.8 tries (133 tests) | NZ GB PH ZA US
AU:  6.9 tries (266 tests) | AU GB PH US CA
GB:  7.1 tries (684 tests) | GB PH US CA JM
KE:  7.1 tries ( 75 tests) | KE TZ ZA GH PH
IN:  7.3 tries (176 tests) | IN LK GB PH SG
MY:  7.4 tries ( 83 tests) | MY US CA PH SG
NG:  7.5 tries ( 70 tests) | NG US GB ZA CA
PH:  7.5 tries ( 72 tests) | PH GB CA US SG
PK:  7.7 tries ( 97 tests) | PK IN BD GB PH
HK:  7.9 tries ( 72 tests) | HK PH US CA MY
US:  8.9 tries (649 tests) | GB US CA PH IN
ZA:  9.0 tries ( 74 tests) | ZA TZ CA JM PH
SG:  9.1 tries ( 86 tests) | SG GB PH AU CA
. . . . . . . . . .

In [12]:
from sklearn.neural_network import MLPClassifier

# Multi-layer Perceptron Model with undersampling
mlp_model = MLPClassifier(random_state=3)
average_performance(mlp_model,X, undersampling=True)
average_performance(mlp_model,X_25, undersampling=True)
average_performance(mlp_model,X_50, undersampling=True)

. . . . . . . . . . 
Average Accuracy: 0.1521212121212121
Averaged Average tries: 7.705049952470591
JM:  2.8 tries ( 70 tests) | JM TZ PH IN SG
TZ:  3.7 tries ( 49 tests) | TZ IN PH AU JM
PH:  4.8 tries ( 72 tests) | PH JM TZ SG HK
SG:  5.4 tries ( 86 tests) | JM SG PH TZ IN
LK:  5.7 tries ( 67 tests) | LK JM PH TZ IN
IN:  5.9 tries (176 tests) | IN JM PH LK TZ
KE:  6.1 tries ( 75 tests) | TZ KE PH JM AU
HK:  6.6 tries ( 72 tests) | HK PH TZ JM IN
CA:  7.1 tries (252 tests) | CA PH JM TZ IN
GB:  7.2 tries (684 tests) | JM PH TZ GB CA
NZ:  7.7 tries (133 tests) | JM NZ PH IN TZ
US:  8.0 tries (649 tests) | PH JM TZ US IN
AU:  8.3 tries (266 tests) | JM PH TZ AU IN
BD:  8.4 tries ( 80 tests) | BD IN JM PH TZ
IE:  9.5 tries (180 tests) | PH TZ IE JM CA
NG: 10.5 tries ( 70 tests) | NG PH JM TZ IN
GH: 10.5 tries ( 65 tests) | TZ PH GH JM IN
MY: 11.9 tries ( 83 tests) | JM PH IN CA SG
PK: 11.9 tries ( 97 tests) | TZ PK PH IN JM
ZA: 12.0 tries ( 74 tests) | JM TZ PH NZ AU
. . . . . . . . . . 

In [14]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression model with scrambling
model = LogisticRegression(max_iter=2000, class_weight='balanced')
average_performance(model, X, scrambling=True)
average_performance(model, X_25, scrambling=True)
average_performance(model, X_50, scrambling=True)

. . . . . . . . . . 
Average Accuracy: 0.1281818181818182
Averaged Average tries: 10.768325244636634
US:  3.5 tries (681 tests) | GB US AU CA IN
GB:  4.3 tries (686 tests) | US GB AU CA IN
AU:  7.2 tries (251 tests) | GB US CA AU IE
CA:  7.4 tries (250 tests) | GB US AU CA NZ
IN:  8.1 tries (167 tests) | US GB CA AU IN
IE:  9.6 tries (175 tests) | GB US AU CA IN
MY: 10.7 tries ( 77 tests) | GB US AU NZ CA
LK: 11.2 tries ( 82 tests) | US GB CA IE AU
NZ: 11.6 tries (133 tests) | US GB AU IE IN
HK: 12.3 tries ( 71 tests) | US GB PK CA AU
NG: 12.4 tries ( 66 tests) | US GB IN IE NZ
TZ: 12.5 tries ( 63 tests) | GB US CA IE LK
PK: 12.6 tries ( 77 tests) | US GB NZ CA IN
ZA: 12.6 tries ( 65 tests) | US GB AU CA IN
GH: 12.7 tries ( 67 tests) | US GB AU CA LK
PH: 12.7 tries ( 90 tests) | US GB AU IE CA
SG: 13.2 tries ( 78 tests) | US GB AU CA IE
KE: 13.5 tries ( 74 tests) | US GB AU CA GH
JM: 13.5 tries ( 67 tests) | US GB GH AU IE
BD: 13.7 tries ( 80 tests) | US GB IN IE CA
. . . . . . . . . .

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


. . . . . . . . . 
Average Accuracy: 0.13272727272727275
Averaged Average tries: 10.54324832200571
GB:  3.9 tries (686 tests) | GB US AU CA IE
US:  4.3 tries (712 tests) | GB US CA AU IE
IN:  7.2 tries (163 tests) | US GB AU IN NZ
CA:  7.4 tries (233 tests) | GB US AU CA NZ
AU:  8.1 tries (243 tests) | GB US CA AU IN
NZ:  8.7 tries (125 tests) | US GB AU IE CA
IE:  8.8 tries (181 tests) | US GB CA AU IE
SG: 10.4 tries ( 75 tests) | GB US CA AU HK
PH: 10.9 tries ( 80 tests) | US GB IE AU NZ
NG: 11.3 tries ( 59 tests) | US GB CA AU NZ
KE: 11.7 tries ( 65 tests) | GB US AU CA NZ
PK: 12.1 tries ( 88 tests) | GB US IN CA IE
MY: 12.2 tries ( 92 tests) | GB US AU ZA IE
HK: 12.8 tries ( 64 tests) | US GB CA AU NZ
LK: 12.8 tries ( 72 tests) | US GB AU CA NG
BD: 12.8 tries ( 72 tests) | US CA GB AU IE
JM: 13.3 tries ( 77 tests) | US GB IE AU IN
ZA: 13.5 tries ( 77 tests) | GB US AU CA LK
GH: 14.2 tries ( 68 tests) | US GB IN NZ AU
TZ: 14.6 tries ( 68 tests) | GB US IN CA NZ


In [17]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier with scrambled labels
rf_model = RandomForestClassifier(random_state=3, class_weight='balanced')
average_performance(rf_model, X, scrambling=True)
average_performance(rf_model, X_25, scrambling=True)
average_performance(rf_model, X_50, scrambling=True)

. . . . . . . . . . 
Average Accuracy: 0.21000000000000002
Averaged Average tries: 10.458215640213723
GB:  1.5 tries (680 tests) | US GB AU CA IN
US:  1.5 tries (680 tests) | US GB AU CA IN
AU:  4.3 tries (260 tests) | GB US AU CA IN
CA:  5.0 tries (230 tests) | US GB AU CA IN
IN:  6.9 tries (170 tests) | US GB AU CA IN
IE:  6.9 tries (180 tests) | GB US AU CA IN
NZ:  9.5 tries (140 tests) | GB US AU CA IN
GH: 12.0 tries ( 70 tests) | US GB AU CA IN
BD: 12.3 tries ( 70 tests) | GB US CA AU IN
LK: 12.8 tries ( 80 tests) | US GB AU CA IN
PK: 12.9 tries ( 90 tests) | US GB AU CA IN
MY: 13.1 tries ( 70 tests) | US GB CA AU NZ
NG: 13.1 tries ( 70 tests) | GB US AU CA IN
KE: 13.2 tries ( 70 tests) | US GB AU CA IE
PH: 13.5 tries ( 80 tests) | US GB AU CA IE
SG: 13.7 tries ( 80 tests) | US GB AU CA IN
HK: 13.7 tries ( 70 tests) | US GB AU CA IE
ZA: 13.8 tries ( 80 tests) | US GB AU CA ZA
JM: 13.9 tries ( 70 tests) | US GB CA AU IN
TZ: 15.5 tries ( 60 tests) | GB US AU CA IE
. . . . . . . . . 

In [18]:
from sklearn.neural_network import MLPClassifier

# Multi-layer Perceptron Model with scrambling
mlp_model = MLPClassifier(random_state=3)
average_performance(mlp_model,X, scrambling=True)
average_performance(mlp_model,X_25, scrambling=True)
average_performance(mlp_model,X_50, scrambling=True)

. . . . . . . . . . 
Average Accuracy: 0.17606060606060608
Averaged Average tries: 10.226453969268865
GB:  3.1 tries (680 tests) | US GB CA AU IE
US:  3.4 tries (680 tests) | US GB CA AU IE
AU:  5.8 tries (260 tests) | GB US CA AU IE
CA:  5.9 tries (230 tests) | US GB AU CA IE
IN:  6.8 tries (170 tests) | GB US AU CA IE
IE:  7.8 tries (180 tests) | GB US CA AU IE
NZ:  8.1 tries (140 tests) | GB US CA IE AU
LK:  8.9 tries ( 80 tests) | GB US CA AU IE
SG: 10.4 tries ( 80 tests) | GB US AU CA IE
MY: 10.4 tries ( 70 tests) | US GB CA AU NG
BD: 11.4 tries ( 70 tests) | US GB CA IE AU
JM: 11.5 tries ( 70 tests) | GB US CA AU IE
GH: 12.3 tries ( 70 tests) | US GB CA AU IN
KE: 13.0 tries ( 70 tests) | GB US IE AU CA
PH: 13.2 tries ( 80 tests) | GB US CA IE IN
TZ: 13.5 tries ( 60 tests) | US GB CA AU IN
NG: 13.6 tries ( 70 tests) | US GB IE AU CA
HK: 14.1 tries ( 70 tests) | US GB AU IE CA
ZA: 15.0 tries ( 80 tests) | US GB CA AU IE
PK: 16.3 tries ( 90 tests) | US GB CA AU IE
. . . . . . . . . 